In [1]:
# Импорт библиотек
import pandas as pd
import os
import random
from math import pi
import torch
from torch.utils.data import random_split, Dataset, DataLoader, TensorDataset
from torch import nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from tkinter import filedialog
import torch.nn.functional as F
from sklearn.metrics import f1_score

In [2]:

# #! Сначала склеить разметку, а потом уже определить разрывы и разметить цифрой 2 начала предупреждающих участков
root = tk.Tk()
root.withdraw()

filePath = filedialog.askopenfilename(filetypes=[("Parquet Files", "*.parquet")])
if filePath:
    print("Выбранный файл:", filePath)
    fileName, fileExtension = os.path.splitext(filePath)
    if fileExtension == '.parquet':
        dfOriginal = pd.read_parquet(filePath)
        df = dfOriginal.copy()
    else:
        print("Выбран неверный файл")
        exit()
else:
    print("Выбор файла отменен.")
    exit()

df['value'] = (df['value'] - df['value'].min()) / (df['value'].max() - df['value'].min())       # Нормализация данных
df.reset_index(drop=True)

Выбранный файл: /home/vest/Desktop/Work/NeuralNetworks/ecg-anomaly-detection/.workspace/Resources/05. Valuated/ECGr1perfect.C.S.V.parquet


,value,edge,time,group
0,0.190297,0.0,0.0,NaN
1,0.164994,0.0,1.0,NaN
2,0.177645,0.0,2.0,NaN
3,0.194514,0.0,3.0,NaN
4,0.160777,0.0,4.0,NaN
...,...,...,...,...
608595,0.000000,0.0,608595.0,NaN
608596,0.000000,0.0,608596.0,NaN
608597,0.000000,0.0,608597.0,NaN
608598,0.000000,0.0,608598.0,NaN


In [3]:
class SecondEcgDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.segmentStarts = df.index[df['edge'] > 0].tolist()
        self.startsCount = len(self.segmentStarts)

        self.cachedTensors = []
        self.cachedLabels = []
        self._get_segment_tensor()
        self.segmentsCount = len(self.cachedTensors)
    
    def _get_segment_tensor(self):
        for index in range(self.startsCount):
            result = 0
            if index + 1 < len(self.segmentStarts):
                if self.df.at[self.segmentStarts[index], 'edge'] != 2:      # По сути условие, что длина отрезка меньше 200
                    segment = self.df.iloc[self.segmentStarts[index]:self.segmentStarts[index+1]]
                    result = 1
            elif self.df.at[self.segmentStarts[index], 'edge'] != 2:        #! Возможно это нужно будет убрать, т.к. тут в выборку может попасть отрезок длиной больше 200
                segment = self.df.iloc[self.segmentStarts[index]:]
                result = 1

            if result:
                segmentValues = segment['value']
                labelValue = ((segment.iloc[0]['edge'] - 2) if segment.iloc[0]['edge'] > 2.0 else 0)
                segmentTensor = torch.tensor(segmentValues.to_numpy(), dtype=torch.float).unsqueeze(1)
                # print(segmentTensor.shape)
                labelTensor = torch.tensor(labelValue, dtype=torch.float).unsqueeze(0)
                # print(labelTensor.shape)
                padding = 200 - len(segmentTensor)
                if padding > 0:
                    segmentTensor = torch.cat((segmentTensor, torch.zeros(padding, 1)), dim=0)
                elif padding < 0:
                    print('В данные зашёл сегмент неверной длины')
                    exit()
                self.cachedTensors.append(segmentTensor)
                self.cachedLabels.append(labelTensor)

    def __len__(self):
        return self.segmentsCount

    def __getitem__(self, index):
        #segmentTensor = self.cachedTensors[index]
        #return segmentTensor
        return self.cachedTensors[index], self.cachedLabels[index]

In [25]:
dataset = SecondEcgDataset(df)
print(len(dataset))
#! Датасет размешивается и разделяется, однако стоит добавить ещё данных и разделить их вручную для более качественного обучения и валидации
trainDataset, valDataset = random_split(dataset, [int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))])
# trainDataset, valDataset, testDataset = random_split(dataset, [int(0.6 * len(dataset)), int(0.2 * len(dataset)), len(dataset) - int(0.6 * len(dataset)) - int(0.2 * len(dataset))])
print(len(trainDataset))
print(len(valDataset))

trainDataloader = torch.utils.data.DataLoader(trainDataset, batch_size=1, shuffle=True)
valDataloader = torch.utils.data.DataLoader(valDataset, batch_size=1, shuffle=False)

5619
5057
562


In [26]:
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, seq_len=200, bidirectional=True):
        super(LSTMAutoencoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1
        add_size = 2*hidden_size if bidirectional else hidden_size

        # LSTM
        self.lstm1 = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
        )

        self.lstm2 = nn.LSTM(
            input_size=add_size,
            hidden_size=hidden_size//2,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
        )

        self.lstm3 = nn.LSTM(
            input_size=add_size//2,
            hidden_size=hidden_size//4,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,
        )

        self.resulter = nn.Linear(seq_len * add_size//4, 1)
        # self.resulter1 = nn.Linear(seq_len * add_size//4, (seq_len * add_size//4)//10)
        # self.resulter2 = nn.Linear((seq_len * add_size//4)//10, (seq_len * add_size//4)//100)
        # self.resulter3 = nn.Linear((seq_len * add_size//4)//100, 1)

        # Sigmoid
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # h1 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size).to(x.device)
        # c1 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size).to(x.device)

        # h2 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size//2).to(x.device)
        # c2 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size//2).to(x.device)

        # h3 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size//4).to(x.device)
        # c3 = torch.zeros(self.num_layers * self.num_directions, x.size(0), self.hidden_size//4).to(x.device)

        # Encoder
        output, _ = self.lstm1(x)#, (h1, c1))       #Output shape: [1, 200, 128]   [batch_size, seq_len, num_directions * hidden_size]
        output, _ = self.lstm2(output)#, (h2, c2))
        # _, (hidden, _) = self.lstm3(output)#, (h0, c0))
        output, _ = self.lstm3(output)#, (h3, c3))
        # print(torch.cat([hidden[-2], hidden[-1]], dim=1).shape)

        # Resulter
        # output = self.resulter(torch.cat([hidden[-2], hidden[-1]], dim=1))      #! Надо попробовать использовать не Hidden, а Output
        batch_size, seq_len, _ = output.size()
        output = output.reshape(batch_size, -1)
        output = self.resulter(output)      #! Надо попробовать использовать не Hidden, а Output
        # output = self.resulter2(output)
        # output = self.resulter3(output)
        # Надо, чтобы он принимал 32*200 значений
        # print(output.shape)

        # Sigmoid
        output = self.sigmoid(output) 

        return output

In [31]:
# Параметры для создания модели
input_size = 1 # Так как ЭКГ - одномерный сигнал
hidden_size = 1024 # Можно изменить в зависимости от сложности задачи и размера данных
num_layers = 1 # Можно изменить в зависимости от сложности задачи

# Создание экземпляра модели
device = ("cuda")
# model = LSTMAutoencoder(input_size, hidden_size, num_layers).to(device)
model = LSTMAutoencoder(input_size, hidden_size, num_layers, 200, True).to(device)
print(model)

def train(trainDataloader, valDataloader, model, criterion, optimizer):
    allTargets = []
    allOutputs = []

    # Обучение модели
    model.train()
    trainPbar = tqdm(trainDataloader, desc="Training")
    for inputs, targets in trainPbar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        allTargets.append(targets.cpu().detach().squeeze().numpy())
        allOutputs.append(outputs.cpu().detach().squeeze().numpy())
        loss.backward()
        optimizer.step()
        trainPbar.set_postfix({'Loss': f'{loss.item()}'})
    allTargets = np.array(allTargets)
    allOutputs = np.array(allOutputs)
    # allTargets[allTargets == 0] = 0
    # allTargets[allTargets == 0.5] = 0
    # allTargets[allTargets == 1] = 1
    # allOutputs[allOutputs == 0] = 0
    # allOutputs[allOutputs == 0.5] = 0
    # allOutputs[allOutputs == 1] = 1
    allTargets = np.round(allTargets).astype(int)
    allOutputs = np.round(allOutputs).astype(int)
    np.set_printoptions(threshold=np.inf)
    f1 = f1_score(allTargets, allOutputs)
    print(f"F1: {f1}")

    f1 = 0
    allTargets = []
    allOutputs = []
    # Валидация модели
    model.eval() # Переводим модель в режим валидации
    valPbar = tqdm(valDataloader, desc="Validating")
    with torch.no_grad(): # Отключаем вычисление градиентов
        for inputs, targets in valPbar:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            valLoss = criterion(outputs, targets)
            allTargets.append(targets.cpu().detach().squeeze().numpy())
            allOutputs.append(outputs.cpu().detach().squeeze().numpy())
            valPbar.set_postfix({'Val_Loss': f'{valLoss.item()}'})
        allTargets = np.array(allTargets)
        allOutputs = np.array(allOutputs)
        # allTargets[allTargets == 0] = 0
        # allTargets[allTargets == 0.5] = 0
        # allTargets[allTargets == 1] = 1
        # allOutputs[allOutputs == 0] = 0
        # allOutputs[allOutputs == 0.5] = 0
        # allOutputs[allOutputs == 1] = 1
        allTargets = np.round(allTargets).astype(int)
        allOutputs = np.round(allOutputs).astype(int)
        f1 = f1_score(allTargets, allOutputs)
        print(f"F1: {f1}")
    
    trainPbar.close()
    valPbar.close()

    return loss.item(), valLoss.item()

LSTMAutoencoder(
  (lstm1): LSTM(1, 1024, batch_first=True, bidirectional=True)
  (lstm2): LSTM(2048, 512, batch_first=True, bidirectional=True)
  (lstm3): LSTM(1024, 256, batch_first=True, bidirectional=True)
  (resulter): Linear(in_features=102400, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [34]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)

ll, vl = 0, 0
for epoch in range(5):
    print(f'Epoch: {epoch} ')
    ll, vl = train(trainDataloader, valDataloader, model, criterion, optimizer)

print('---------------------------------------------------------------------------------------------')
print('Done!')

Epoch: 0 


Training: 100%|██████████| 5057/5057 [06:30<00:00, 12.95it/s, Loss=0.0362028032541275]   


F1: 0.5032258064516129


Validating: 100%|██████████| 562/562 [00:19<00:00, 28.86it/s, Val_Loss=0.01543779019266367] 


F1: 0.47058823529411764
Epoch: 1 


...

F1: 0.5882352941176471
Epoch: 4 


Training: 100%|██████████| 5057/5057 [06:33<00:00, 12.86it/s, Loss=0.02517802082002163]  


F1: 0.525679758308157


Validating: 100%|██████████| 562/562 [00:19<00:00, 29.17it/s, Val_Loss=0.02135414630174637] 

F1: 0.6285714285714286
---------------------------------------------------------------------------------------------
Done!


In [29]:
print('Loss: ', ll)
print('ValLoss: ', vl)

Loss:  0.2790231704711914
ValLoss:  0.2800871431827545


In [35]:
# Сохранение модели
torch.save(model.state_dict(), '../.workspace/Models/SecondStageModel.53.pt')